## Install Package


In [ ]:
# install package using package installer for python(pip)


#install youtube-dl
!pip install --upgrade youtube-dl

# Test yt-dlp package
!pip install -U yt-dlp

#install openCV
!pip install opencv-python

#install videointelligence
!pip install ipython google-cloud-videointelligence

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import pacakges


import os
import time
import youtube_dl
import yt_dlp
#from google.cloud import videointelligence
from google.cloud import storage

In [ ]:
# mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import package

import pandas as pd
import numpy as np

## Export Data from Google Ads Platform

In [ ]:
# read dataset

# list of missing values
missing_values = [' --']

# video mapping
yt = pd.read_csv('/content/drive/MyDrive/HEB campaign data/HEB_YouTube - Video Mapping.csv', na_values = missing_values)
tv = pd.read_csv('/content/drive/MyDrive/HEB campaign data/Spark TV - Video Mapping.csv', na_values = missing_values)

# conversion reporting
yt_conversion = pd.read_csv('/content/drive/MyDrive/HEB campaign data/HEB_YouTube - Conversion Reporting.csv', na_values = missing_values)
tv_conversion = pd.read_csv('/content/drive/MyDrive/HEB campaign data/Spark TV - Conversion Reporting.csv', na_values = missing_values)

# campaign performance reporting
yt_performance = pd.read_csv('/content/drive/MyDrive/HEB campaign data/HEB_YouTube - Campaign Performance Reporting.csv', na_values = missing_values)
tv_performance = pd.read_csv('/content/drive/MyDrive/HEB campaign data/Spark TV - Campaign Performance Reporting.csv', na_values = missing_values)

In [ ]:
# Concatenate tables

HEB_video_list = pd.concat([yt,tv])
HEB_conversion_report = pd.concat([yt_conversion, tv_conversion])
HEB_performance_report = pd.concat([yt_performance, tv_performance])

In [ ]:
HEB_video_list = HEB_video_list.reset_index()
HEB_video_list.drop(columns='index', axis=1, inplace=True)
HEB_video_list.tail()

In [ ]:
HEB_video_list.info()

In [ ]:
# drop the previous 'Video ID' column since we're going to extract it from URL

HEB_video_list.drop('Video ID', axis=1, inplace = True)

In [ ]:
# count NaN in 'Video'

count_nan = HEB_video_list['Video'].isna().sum()
row = len(HEB_video_list.index)

print('There are ' + str(row) + ' creative records in video mapping report')
print('In the video mapping report, ' + str(count_nan) + ' records are missing Youtube video link')
print('In the video mapping report, ' + str(row-count_nan) + ' records have Youtube video link')
print('Records with missing video links as a percentage of total records are: ' + str(count_nan/row))

In [ ]:
df = HEB_video_list[['Video']]
df = df.dropna(subset=['Video'])
df

In [ ]:
import urllib.parse as urlparse

def video_id(value):
    """
    Examples:
    - http://youtu.be/SA2iWivDJiE
    - http://www.youtube.com/watch?v=_oPAwA_Udwc&feature=feedu
    - http://www.youtube.com/embed/SA2iWivDJiE
    - http://www.youtube.com/v/SA2iWivDJiE?version=3&amp;hl=en_US
    """
    query = urlparse.urlparse(value)
    if query.hostname == 'youtu.be':
        return query.path[1:]
    if query.hostname in ('www.youtube.com', 'youtube.com'):
        if query.path == '/watch':
            p = urlparse.parse_qs(query.query)
            return p['v'][0]
        if query.path[:7] == '/embed/':
            return query.path.split('/')[2]
        if query.path[:3] == '/v/':
            return query.path.split('/')[2]
    # fail?
    return None

In [ ]:
df['Video ID'] = df['Video'].apply(video_id)

In [ ]:
df

In [ ]:
HEB_video_list = HEB_video_list.join(df['Video ID'], how='left')
HEB_video_list

In [ ]:
# save as csv and upload to the BigQuery using Google Sheets

HEB_video_list.to_csv('HEB_video_list.csv')
HEB_conversion_report.to_csv('HEB_conversion.csv')
HEB_performance_report.to_csv('HEB_performance.csv')

In [ ]:
# remove duplicate Video IDs

df_1 = df[['Video ID']].drop_duplicates(keep='first')
df_1

In [ ]:
print('There are '+ str(len(df_1))+ ' unique video IDs in total')

In [ ]:
# save Video ID as txt

df_1.to_csv('video_ids.txt', header=None, index=None, sep='\t', mode='a')

## Download HEB Youtube Video

In [ ]:
# authenticate google cloud platform and google drive
# project has already been created


from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'med-heb-intern-gcp-dev-mg' # Specify your GCP project ID
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
# define function for downloading video from Youtube and uploading it to google cloud storage



# download a Youtube video from a specified url
def downloadYTVideo(video_id):
    # options: output file name and format
    ydl_opts ={'outtmpl': '%(id)s.%(ext)s','format':'mp4'}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download(['https://www.youtube.com/watch?v='+video_id])
    # return the filename
    return video_id+".mp4"

# upload video file to GCP storage bucket
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    #destination_blob_name = hash(source_file_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )


# upload videos to Google Drive folder
def upload_drive(folder_id, source_file_name):
    """Uploads a file to the folder."""
    gfile = drive.CreateFile({'parents': [{'id': folder_id}]})
    # Read file and set it as the content of this instance.
    gfile.SetContentFile(source_file_name)
    gfile.Upload() # Upload the file.


# delete the video file from Jupyter 
def deleteVideoFile(file_name):
    os.remove(file_name)



# YouTube --> GCS pipeline
def runVideoPipeline(video_id,gcs_video_bucket,counter):
    
    # get video from YouTube
    video_file = downloadYTVideo(video_id)
    
    # upload video to GCS storage bucket
    upload_blob(gcs_video_bucket, video_file, destination_blob_name = str(counter)+'_'+video_id+'.mp4')
    #upload_blob(gcs_video_bucket, video_file, destination_blob_name = hash(video_id)+'.mp4')
    
    # delete video from Jupyter storage
    deleteVideoFile(video_file)

### Download Video Bucket by Bucket

In [ ]:
# params

gcs_video_bucket = 'all_videos_test' #'name_of_gcs_bucket_to_store_videos'

In [ ]:
# Upload 'video_ids.txt' from local machine

from google.colab import files
uploaded = files.upload()

Saving final_video_ids.txt to final_video_ids.txt


In [ ]:
# get list of YT videos to analyze from text file

with open('/content/final_video_ids.txt') as f:
    video_ids = f.read().splitlines()

    private = [] 

    # Counter for vid name
    counter = 1

    for vid in video_ids:
        if vid != 'video_id':
            video_id = vid
            try:
              runVideoPipeline(video_id,gcs_video_bucket,counter)
              counter += 1 
            except:
               private.append(vid)
               # store the vid of private youtube video


### Check List of Private Videos




In [ ]:
# see the output of private video id stored in the list

private

In [ ]:
# save the private video id as a txt file

private_vid = pd.DataFrame (private, columns = ['private_vid'])

private_vid.to_csv('private_vid.txt', header=None, index=None, sep='\t', mode='a')

In [ ]:
# count total# of private videos

print('A total of ' + str(len(private)) + ' YouTube videos are private.')
print('To download these private videos, you need to sign in and make sure you have been granted access to it.')

In [ ]:
unique_vid_total = len(df_1)
private_vid_total = len(private)
downloadable_vid_total = len(df_1) - len(private)
private_percent = len(private)/len(df_1)

In [ ]:
# count total# of videos that can be download

print('A total of ' + str(downloadable_vid_total) + ' YouTube videos are downloadable.')
print('The proportion of downloadable YouTube videos is ' + str(1-private_percent) + ' .')